In [335]:
import json
import numpy as np
import pandas as pd
import plotly.express as px
import kaleido

In [350]:
# Select start date
start_date = "2022-01-01"
# Input current price of ETH
ethusd = 2683
# Initialize collection slugs
slugs = [
#     "anticyclone-by-william-mapan",
#     "flux-by-owen-moore",
#     "ancient-courses-of-fictional-rivers-by-robert-hodg",
#     "memories-of-qilin-by-emily-xie",
#     "entretiempos-by-marcelo-soria-rodriguez",
#     "para-bellum-by-matty-mariansky",
#     "screens-by-thomas-lin-pedersen",
#     "cosmic-reef-by-leo-villareal",
#     "chimera-by-mpkoz",
#     "jiometory-no-compute-ziometori-ha-keisansaremasen",
#     "vortex-by-jen-stark",
#     "gazers-by-matt-kane",
#     "bent-by-ippsketch",
#     "autology-by-steganon",
#     "asemica-by-emily-edelman-dima-ofman-andrew-badr",
#     "edifice-by-ben-kovach",
#     "skulptuur-by-piter-pasma",
#     "fragments-of-an-infinite-field-by-monica-rizzolli",
#     "trossets-by-anna-carreras",
#     "geometry-runners-by-rich-lord-1",
#     "phase-by-loren-bednar",
#     "scribbled-boundaries-by-william-tan",
#     "pigments-by-darien-brito",
#     "endless-nameless-by-rafael-rozendaal",
#     "glitch-crystal-monsters-by-alida-sun",
#     "century-by-casey-reas",
#     "dreams-by-joshua-bagley",
#     "the-blocks-of-art-by-shvembldr",
#     "frammenti-by-stefano-contiero",
#     "algorhythms-by-han-x-nicolas-daniel",
#     "bubble-blobby-by-jason-ting",
#     "watercolor-dreams-by-numbersinmotion",
#     "subscapes-by-matt-deslauriers",
#     "elementals-by-michael-connolly",
#     "algobots-by-stina-jones",
#     "synapses-by-chaosconstruct",
#     "aerial-view-by-dalenz",
#     "inspirals-by-radix",
#     "apparitions-by-aaron-penne",
#     "720-minutes-by-alexis-andre",
#     "archetype-by-kjetil-golid",
#    "27-bit-digital-by-kai",
#     "spectron-by-simon-de-mai",
#     "unigrids-by-zeblocks",
#     "hyperhash-by-beervangeer",
#     "nimbuds-by-bryan-brinkman",
#     "ignition-by-ge1doot",
#     "singularity-by-hideki-tsukamoto",
#     "elevated-deconstructions-by-luxpris",
#     "cryptoblots-by-daim-aggott-honsch",
#     "dynamic-slices-by-pxlq-1",
#     "chromie-squiggle-by-snowfro",
#     "genesis-by-dca",
#     "construction-token-by-jeff-davis"
#     "ringers-by-dmitri-cherniak",
#     "fidenza-by-tyler-hobbs"
]

In [351]:
# Iterate through collections
for slug in slugs:
    print(slug)
    # Ingest price history json file
    with open(f'price_histories/{slug}.json', 'r') as f:
        data = json.load(f)

    # Extract price history, and convert it to a NumPy array
    price_history = np.array(data["price_history"])
    # Ingest price history into a dataframe
    df = pd.DataFrame(price_history, columns = ['event_time','eth_price','usd_price'])
    # Clean event times
    df["event_time"] = df["event_time"].apply(lambda x : x.split("T")[0])
    # Convert eth_price and usd_price to int
    df["eth_price"] = df["eth_price"].apply(lambda x : float(x))
    df["usd_price"] = df["usd_price"].apply(lambda x : float(x))
    # Filter for events after start_date
    df_daterange = df[df["event_time"] > start_date]
    
    # Create bins
    counts, bins = np.histogram(
        df_daterange.eth_price,
        bins=np.arange(0,
                       int(max(df_daterange.eth_price)*1.1),
                       max(0.1, int(df_daterange.eth_price.mean()/1))
                      ))
    bins = 0.5 * (bins[:-1] + bins[1:])

    fig = px.bar(x=bins,
                 y=counts,
                 labels={'x':'Price (ETH)',
                         'y':'Count'},
                 title=f"{slug} sale price history since {start_date}"
                )

    fig.add_vline(x=float(data["floor_price"]),
                  line_width=3,
                  line_dash="dash",
                  line_color="red",
                  annotation_text="Floor",
                  annotation_position="top",
                 )

    fig.add_vline(x=float(df_daterange["eth_price"].mean()),
                  line_width=3,
                  line_dash="dash",
                  line_color="green",
                  annotation_text="Avg",
                  annotation_position="top"
                 )

    fig.write_image(f"price_charts/{slug}-eth.png")
    #fig.show()
    
    # Create bins
    counts, bins = np.histogram(
        df_daterange.usd_price,
        bins=np.arange(0,
                   int(max(df_daterange.usd_price)*1.1),
                   int((df_daterange.usd_price).mean()/1)
                  ))
    bins = 0.5 * (bins[:-1] + bins[1:])

    fig = px.bar(x=bins,
                 y=counts,
                 labels={'x':'Price (USD)',
                         'y':'Count'},
                 title=f"{slug} sale price history since {start_date}")

    fig.add_vline(x=float(data["floor_price"])*ethusd,
                  line_width=3,
                  line_dash="dash",
                  line_color="red",
                  annotation_text="Floor",
                  annotation_position="top",
                 )

    fig.add_vline(x=float(df_daterange["usd_price"].mean()),
                  line_width=3,
                  line_dash="dash",
                  line_color="green",
                  annotation_text="Avg",
                  annotation_position="top"
                 )

    fig.write_image(f"price_charts/{slug}-usd.png")
    #fig.show()